## A. Market Research (Sale of Oatmeal Cookies)

In [ ]:
# Mount your gdrive for colab
from google.colab import drive
drive.mount('/content/drive')

# import the pandas library
import pandas as pd 
# read the Excel file 'Reviews_oatmeal_cookies.xlsx' from Google Drive
dat = pd.read_excel('drive/MyDrive/oat_review/Reviews_oatmeal_cookies.xlsx')

# display the first three rows of the data
dat.head(3)

In [ ]:
# display information about the dataset
dat.info()

In [ ]:
# find the mean rating
dat.Score.mean()

In [ ]:
# Import the WordCloud and Matplotlib libraries
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Join the 'Summary' column of 'dat' where the 'Score' is less than or equal to 2
text = ' '.join(dat[dat.Score <=2].Summary)

# Generate a WordCloud object with specified parameters
wordcloud = WordCloud(width=800, height=800, background_color='white', min_font_size=10).generate(text)

# Create a figure to display the WordCloud object
plt.figure(figsize=(8, 8), facecolor=None)

# Display the WordCloud object
plt.imshow(wordcloud)

# Remove the axis ticks and labels
plt.axis('off')

# Adjust the layout to remove any extra whitespace
plt.tight_layout(pad=0)

# Display the figure
plt.show()

In [ ]:
# Join all reviews with line break seperator
text1 = '/n'.join(dat.Text)

# Open a text file in write mode and save it
with open('/content/drive/MyDrive/text_review.txt', 'w') as file:
    file.write(text1)

# open the file in read mode
with open('/content/drive/MyDrive/text_review.txt', 'r') as file:

    # read the contents of the file
    contents = file.read()

    # split the contents into words
    words = contents.split()

    # count the number of words
    num_words = len(words)

# print the number of words
print("The file contains", num_words, "words.")

In [6]:
# Upgrade pip
# ! pip install --upgrade pip

# install farm-haystack along with with its dependencies for google colab environment
# ! pip install farm-haystack[colab]

# Import the TextConverter to convert text to documents
from haystack.nodes import TextConverter

# Set the directory path for the input text file
DOC_DIR = '/content/drive/MyDrive/text_review.txt'

# Initialize a TextConverter object with the specified parameters
converter = TextConverter(
    remove_numeric_tables=True,  # Remove numeric tables from the text
    valid_languages=["en"]  # Specify that the text is in English
)

# Convert the text file to a list of Haystack Document objects and extract metadata
# The convert() method returns a tuple with the list of documents and the metadata dictionary
docs = converter.convert(file_path=DOC_DIR, meta=None)[0]

In [ ]:
# Importing PreProcessor class from the haystack.nodes module
from haystack.nodes import PreProcessor

# Creating an instance of the PreProcessor class with various options
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_overlap=3,
    split_respect_sentence_boundary=False,
)

# Using the process method of the preprocessor instance to process the docs
processed_docs = preprocessor.process(docs)

In [9]:
# install farm-haystack along with with its dependencies for faiss
# NB: You may need to restart the run time after installation before importing FAISSDocumentStore

# !pip install farm-haystack[faiss]

# Importing the FAISSDocumentStore class from the haystack.document_stores module
from haystack.document_stores import FAISSDocumentStore

# Creating an instance of the FAISSDocumentStore class with specified options
document_store = FAISSDocumentStore(
    faiss_index_factory_str="Flat",
    embedding_dim=1536
)

In [ ]:
# delete documents in database
document_store.delete_documents()

# add preprocessed document
document_store.write_documents(processed_docs)

In [ ]:
# Import the EmbeddingRetriever class from the haystack.nodes module
# To get OpenAI API key: https://openai.com/product

from haystack.nodes import EmbeddingRetriever

# Set the OpenAI API key
MY_API_KEY = "enter api key from open ai"

# Initialize an EmbeddingRetriever object with the specified parameters
retriever = EmbeddingRetriever(
    document_store=document_store,  # Document database
    embedding_model="text-embedding-ada-002",  # Pre-trained text embedding model
    batch_size=32,  # Batch size for processing data
    api_key=MY_API_KEY,  # OpenAI API key for authentication
    max_seq_len=1024  # Maximum length of input sequences
)

# Update the embeddings of documents in the document store using the retriever object
document_store.update_embeddings(retriever)

In [12]:
# Import the OpenAIAnswerGenerator class from the haystack.nodes module
from haystack.nodes import OpenAIAnswerGenerator

# Initialize an OpenAIAnswerGenerator object with the specified parameters
generator = OpenAIAnswerGenerator(
    api_key=MY_API_KEY,  # OpenAI API key for authentication
    model="text-davinci-003",  # Open AI text model
    temperature=.5,  # Controls the randomness of the generated responses
    max_tokens=100  # Maximum number of tokens (words) in the generated responses
)

In [13]:
# Import the GenerativeQAPipeline class from the haystack.pipelines module
from haystack.pipelines import GenerativeQAPipeline

# Initialize a GenerativeQAPipeline object with the specified parameters
gpt_search_engine = GenerativeQAPipeline(
    generator=generator,  # Answer generator object
    retriever=retriever  # Retriever object
)

### Query 1: What are some complaints about the cookies?

In [ ]:
# Import the print_answers function from the haystack.utils module
from haystack.utils import print_answers

# Set the input query string
query_input = "What are some complaints about the cookies?"
query = query_input

# Define the search parameters as a dictionary with retriever and generator parameters
params = {
    "Retriever": {"top_k": 15},  # Retrieve the top 15 most relevant documents
    "Generator": {"top_k": 1}  # Generate the top 1 most likely answer
}

# Use the GenerativeQAPipeline object to answer the input query with the specified search parameters
answer = gpt_search_engine.run(query=query, params=params)

# Print the predicted answers with minimal details
print_answers(answer, details="minimum")

### Query 2: What else are people saying about the cookie's dryness?

In [ ]:
# Import the print_answers function from the haystack.utils module
from haystack.utils import print_answers

# Set the input query string
query_input = "What else are people saying about the cookie's dryness?"
query = query_input

# Define the search parameters as a dictionary with retriever and generator parameters
params = {
    "Retriever": {"top_k": 15},  # Retrieve the top 15 most relevant documents
    "Generator": {"top_k": 1}  # Generate the top 1 most likely answer
}

# Use the GenerativeQAPipeline object to answer the input query with the specified search parameters
answer = gpt_search_engine.run(query=query, params=params)

# Print the predicted answers with minimal details
print_answers(answer, details="minimum")

### Query 3: What are some customer recommendations to improve the product?

In [ ]:
# Import the print_answers function from the haystack.utils module
from haystack.utils import print_answers

# Set the input query string
query_input = "What are some customer recommendations to improve the product?"
query = query_input

# Define the search parameters as a dictionary with retriever and generator parameters
params = {
    "Retriever": {"top_k": 15},  # Retrieve the top 15 most relevant documents
    "Generator": {"top_k": 1}  # Generate the top 1 most likely answer
}

# Use the GenerativeQAPipeline object to answer the input query with the specified search parameters
answer = gpt_search_engine.run(query=query, params=params)

# Print the predicted answers with minimal details
print_answers(answer, details="minimum")

## B. Inquiry System (UK Net Zero Policy Documents)

In [1]:
# Install the 'farm-haystack' package with PDF support
# ! pip install 'farm-haystack[pdf]'

# Import the PDFToTextConverter class from haystack.nodes module
from haystack.nodes import PDFToTextConverter

# Import the convert_files_to_docs function from haystack.utils module
from haystack.utils import convert_files_to_docs

# Set the directory path where the documents to be converted are located
DOC_DIR = '/content/drive/MyDrive/net_zero'

# Use the convert_files_to_docs function to convert the PDF documents in the directory to text
# The split_paragraphs parameter specifies whether to split the text into paragraphs or not
docs = convert_files_to_docs(dir_path=DOC_DIR, split_paragraphs=True)

In [ ]:
# Importing PreProcessor class from the haystack.nodes module
from haystack.nodes import PreProcessor

# Creating an instance of the PreProcessor class with various options
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_overlap=3,
    split_respect_sentence_boundary=False,
)

# Using the process method of the preprocessor instance to process the docs
processed_docs = preprocessor.process(docs)

# install farm-haystack along with with its dependencies for faiss
# !pip install farm-haystack[faiss]

# Importing the FAISSDocumentStore class from the haystack.document_stores module
from haystack.document_stores import FAISSDocumentStore

# Creating an instance of the FAISSDocumentStore class with specified options
# NB: Remove the previous faiss_document_store.db from the previous project or run this section in a new directory

document_store = FAISSDocumentStore(
    faiss_index_factory_str="Flat",
    embedding_dim=1536
)

# delete documents in database
document_store.delete_documents()

# add preprocessed document
document_store.write_documents(processed_docs)

# Import the EmbeddingRetriever class from the haystack.nodes module
from haystack.nodes import EmbeddingRetriever

# Set the OpenAI API key
MY_API_KEY = "enter api key from open ai"

# Initialize an EmbeddingRetriever object with the specified parameters
retriever = EmbeddingRetriever(
    document_store=document_store,  # Document database
    embedding_model="text-embedding-ada-002",  # Pre-trained text embedding model
    batch_size=32,  # Batch size for processing data
    api_key=MY_API_KEY,  # OpenAI API key for authentication
    max_seq_len=1024  # Maximum length of input sequences
)

# Update the embeddings of documents in the document store using the retriever object
document_store.update_embeddings(retriever)

# Import the OpenAIAnswerGenerator class from the haystack.nodes module
from haystack.nodes import OpenAIAnswerGenerator

# Initialize an OpenAIAnswerGenerator object with the specified parameters
generator = OpenAIAnswerGenerator(
    api_key=MY_API_KEY,  # OpenAI API key for authentication
    model="text-davinci-003",  # Open AI text model
    temperature=.5,  # Controls the randomness of the generated responses
    max_tokens=100  # Maximum number of tokens (words) in the generated responses
)

# Import the GenerativeQAPipeline class from the haystack.pipelines module
from haystack.pipelines import GenerativeQAPipeline

# Initialize a GenerativeQAPipeline object with the specified parameters
gpt_search_engine = GenerativeQAPipeline(
    generator=generator,  # Answer generator object
    retriever=retriever  # Retriever object
)

### Query 1: What does net zero even mean?

In [ ]:
# Import the print_answers function from the haystack.utils module
from haystack.utils import print_answers

# Set the input query string
query_input = "What does net zero even mean?"
query = query_input

# Define the search parameters as a dictionary with retriever and generator parameters
params = {
    "Retriever": {"top_k": 15},  # Retrieve the top 15 most relevant documents
    "Generator": {"top_k": 1}  # Generate the top 1 most likely answer
}

# Use the GenerativeQAPipeline object to answer the input query with the specified search parameters
answer = gpt_search_engine.run(query=query, params=params)

# Print the predicted answers with minimal details
print_answers(answer, details="minimum")

### Query 2: When does the UK intend to reach net zero? 

In [ ]:
# Import the print_answers function from the haystack.utils module
from haystack.utils import print_answers

# Set the input query string
query_input = "When does the UK intend to reach net zero?"
query = query_input

# Define the search parameters as a dictionary with retriever and generator parameters
params = {
    "Retriever": {"top_k": 15},  # Retrieve the top 15 most relevant documents
    "Generator": {"top_k": 1}  # Generate the top 1 most likely answer
}

# Use the GenerativeQAPipeline object to answer the input query with the specified search parameters
answer = gpt_search_engine.run(query=query, params=params)

# Print the predicted answers with minimal details
print_answers(answer, details="minimum")

### Query 3: I am saving to buy a new petrol car in 2038, is there any information about how feasible this may be in that year?

In [ ]:
# Import the print_answers function from the haystack.utils module
from haystack.utils import print_answers

# Set the input query string
query_input = "I am saving to buy a new petrol car in 2038, is there any information about how feasible this may be in that year?"
query = query_input

# Define the search parameters as a dictionary with retriever and generator parameters
params = {
    "Retriever": {"top_k": 15},  # Retrieve the top 15 most relevant documents
    "Generator": {"top_k": 1}  # Generate the top 1 most likely answer
}

# Use the GenerativeQAPipeline object to answer the input query with the specified search parameters
answer = gpt_search_engine.run(query=query, params=params)

# Print the predicted answers with minimal details
print_answers(answer, details="minimum")

### Query 4: The government will take away my diesel car in 2030. This has been my only car for 10 years and I will hate it if the government takes it away

In [ ]:
# Import the print_answers function from the haystack.utils module
from haystack.utils import print_answers

# Set the input query string
query_input = "The government will take away my diesel car in 2030. This has been my only car for 10 years and I will hate it if the government takes it away"
query = query_input

# Define the search parameters as a dictionary with retriever and generator parameters
params = {
    "Retriever": {"top_k": 15},  # Retrieve the top 15 most relevant documents
    "Generator": {"top_k": 1}  # Generate the top 1 most likely answer
}

# Use the GenerativeQAPipeline object to answer the input query with the specified search parameters
answer = gpt_search_engine.run(query=query, params=params)

# Print the predicted answers with minimal details
print_answers(answer, details="minimum")

### Query 5: What information is there about the large fans that would cool the atmosphere and stop climate change?

In [ ]:
# Import the print_answers function from the haystack.utils module
from haystack.utils import print_answers

# Set the input query string
query_input = "What information is there about the large fans that would cool the atmosphere and stop climate change?"
query = query_input

# Define the search parameters as a dictionary with retriever and generator parameters
params = {
    "Retriever": {"top_k": 15},  # Retrieve the top 15 most relevant documents
    "Generator": {"top_k": 1}  # Generate the top 1 most likely answer
}

# Use the GenerativeQAPipeline object to answer the input query with the specified search parameters
answer = gpt_search_engine.run(query=query, params=params)

# Print the predicted answers with minimal details
print_answers(answer, details="minimum")